# Ground state search on a tree

In [1]:
using NamedGraphs
using ITensors
using ITensorNetworks
using ITensorUnicodePlots
using Dictionaries
using SparseArrayKit

include("utils.jl")

In this notebook we perform a ground state search for a spin model on a lattice with a tree-like geometry using the [density matrix renormalization group (DMRG)](https://tensornetwork.org/mps/algorithms/dmrg/) algorithm.

As an example, we will consider a next-to-nearest neighbor spin-1/2 Heisenberg model with Hamiltonian

$$
H = J_1 \sum_{\langle i, j \rangle} \vec{S}_i \cdot \vec{S}_j + J_2 \sum_{\langle\langle i, j \rangle\rangle} \vec{S}_i \cdot \vec{S}_j + h \sum_i S^z_i,
$$

on a spin system defined on a small comb tree with 12 sites
<center><img align="center" width="300" src="./fig/medium_comb_tree.svg"></center>

## Tree tensor networks

Having introduced the general `ITensorNetwork` interface in the [ITensorNetworks.jl introduction](./itensornetworks_demo.ipynb), we now specialize to tree tensor networks. A `TreeTensorNetwork` is nothing more than an `ITensorNetwork` on a tree graph that keeps track of its orthogonality center. We can construct a random tree tensor network state with bond dimension 5 on the geometry shown above in the following way:

In [2]:
dims = (4, 3)
c = named_comb_tree(dims)
s = siteinds("S=1/2", c)
root_vertex = (1, 3)
χ = 5

ψ = random_ttn(ComplexF64, s; link_space=χ)
@visualize ψ

    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ψ(2, 3)⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀25⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ψ(2, 2)⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀2⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ψ(3, 3)⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈5⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡠5⠀2⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀5ψ(2, 1)⠀⠀⠀⠀⠀⠀ψ(3, 2)⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⢀⣀ψ(1, 2)ψ(1, 1)⠀⠀⠀⠀2⠀⠀⠈5⠒ψ(3, 1)⠘⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀ψ(1, 3)⠀2⠀⠀⠀⠀⠀⠀⠘⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀2⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀5⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ψ(4, 1)⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀2⠀⠑5⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ψ(4, 2)⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀2⠑5⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

TreeTensorNetwork{Tuple{Int64, Int64}} with 12 vertices:
12-element Vector{Tuple{Int64, Int64}}:
 (1, 1)
 (2, 1)
 (3, 1)
 (4, 1)
 (1, 2)
 (2, 2)
 (3, 2)
 (4, 2)
 (1, 3)
 (2, 3)
 (3, 3)
 (4, 3)

and 11 edge(s):
(1, 1) => (2, 1)
(1, 1) => (1, 2)
(2, 1) => (3, 1)
(2, 1) => (2, 2)
(3, 1) => (4, 1)
(3, 1) => (3, 2)
(4, 1) => (4, 2)
(1, 2) => (1, 3)
(2, 2) => (2, 3)
(3, 2) => (3, 3)
(4, 2) => (4, 3)

with vertex data:
12-element Dictionary{Tuple{Int64, Int64}, Any}
 (1, 1) │ ((dim=2|id=54|"S=1/2,Site,n=1×1"), (dim=5|id=993|"1×1,2×1"), (dim=5|i…
 (2, 1) │ ((dim=2|id=707|"S=1/2,Site,n=2×1"), (dim=5|id=993|"1×1,2×1"), (dim=5|…
 (3, 1) │ ((dim=2|id=502|"S=1/2,Site,n=3×1"), (dim=5|id=35|"2×1,3×1"), (dim=5|i…
 (4, 1) │ ((dim=2|id=408|"S=1/2,Site,n=4×1"), (dim=5|id=795|"3×1,4×1"), (dim=5|…
 (1, 2) │ ((dim=2|id=621|"S=1/2,Site,n=1×2"), (dim=5|id=460|"1×1,1×2"), (dim=5|…
 (2, 2) │ ((dim=2|id=866|"S=1/2,Site,n=2×2"), (dim=5|id=583|"2×1,2×2"), (dim=5|…
 (3, 2) │ ((dim=2|id=402|"S=1/2,Site,n=3×2"), (dim

A randomly initialized tree tensor network state does not have a well-defined orthogonality center. We can move the orthogonality center by calling the `ITensors.orthogonalize` method which gauges the state around a given vertex:

In [3]:
@show ortho_center(ψ);
ψ = orthogonalize(ψ, (2, 1))
@show ortho_center(ψ);

ortho_center(ψ) = [(1, 1), (2, 1), (3, 1), (4, 1), (1, 2), (2, 2), (3, 2), (4, 2), (1, 3), (2, 3), (3, 3), (4, 3)]
ortho_center(ψ) = [(2, 1)]


## The Hamiltonian as a tree tensor network

In order to apply the DMRG algorithm (or any DMRG-like sweeping routine for that matter) we need to write the system Hamiltonian as a tensor network operator with the same geometry as the corresponding state. This is done by first constructing a lazy algebraic representation of the Hamiltonian using the familiar `ITensors.OpSum` interface, and then converting it to a `TreeTensorNetwork` in an automated way.

Constructing the Hamiltonian in an algebraic `OpSum` representation can be done in exactly the same way as you normally would (see for example the [ITensors.jl DMRG tutorial](https://itensor.github.io/ITensors.jl/dev/tutorials/DMRG.html)), where the usual integer site labels are now replaced by graph vertex labels of the appropriate type (in this case tuples of integers). The Hamiltonian defined above (also exported through the `ITensorNetworks.heisenberg` method) can be constructed as:

In [4]:
J1 = -1.0
J2 = 2
h = -0.2

Hos = OpSum()
for (v1, v2) in nearest_neighbors(c)
    Hos += J1 / 2, "S+", v1, "S-", v2
    Hos += J1 / 2, "S-", v1, "S+", v2
    Hos += J1, "Sz", v1, "Sz", v2
end
for (v1, v2) in next_nearest_neighbors(c)
    Hos += J2 / 2, "S+", v1, "S-", v2
    Hos += J2 / 2, "S-", v1, "S+", v2
    Hos += J2, "Sz", v1, "Sz", v2
end
for v in vertices(c)
    Hos += h, "Sz", v
end
Hos

sum(
  -0.5 S+((1, 1),) S-((2, 1),)
  -0.5 S-((1, 1),) S+((2, 1),)
  -1.0 Sz((1, 1),) Sz((2, 1),)
  -0.5 S+((1, 1),) S-((1, 2),)
  -0.5 S-((1, 1),) S+((1, 2),)
  -1.0 Sz((1, 1),) Sz((1, 2),)
  -0.5 S+((2, 1),) S-((3, 1),)
  -0.5 S-((2, 1),) S+((3, 1),)
  -1.0 Sz((2, 1),) Sz((3, 1),)
  -0.5 S+((2, 1),) S-((2, 2),)
  -0.5 S-((2, 1),) S+((2, 2),)
  -1.0 Sz((2, 1),) Sz((2, 2),)
  -0.5 S+((3, 1),) S-((4, 1),)
  -0.5 S-((3, 1),) S+((4, 1),)
  -1.0 Sz((3, 1),) Sz((4, 1),)
  -0.5 S+((3, 1),) S-((3, 2),)
  -0.5 S-((3, 1),) S+((3, 2),)
  -1.0 Sz((3, 1),) Sz((3, 2),)
  -0.5 S+((4, 1),) S-((4, 2),)
  -0.5 S-((4, 1),) S+((4, 2),)
  -1.0 Sz((4, 1),) Sz((4, 2),)
  -0.5 S+((1, 2),) S-((1, 3),)
  -0.5 S-((1, 2),) S+((1, 3),)
  -1.0 Sz((1, 2),) Sz((1, 3),)
  -0.5 S+((2, 2),) S-((2, 3),)
  -0.5 S-((2, 2),) S+((2, 3),)
  -1.0 Sz((2, 2),) Sz((2, 3),)
  -0.5 S+((3, 2),) S-((3, 3),)
  -0.5 S-((3, 2),) S+((3, 3),)
  -1.0 Sz((3, 2),) Sz((3, 3),)
  -0.5 S+((4, 2),) S-((4, 3),)
  -0.5 S-((4, 2),) S+((4, 3),)
  -

This procedure can be adapted to construct Hamiltonians with interactions of arbitrary weight and range. More convenient helper functions like `nearest_neighbors` and `next_nearest_neighbors` will be added to ITensorNetworks.jl in the near future to facilitate this.

We can automatically construct the Hamiltonian as a tree tensor network operator by passing this `OpSum` along with the physical site indices to the tree tensor network constructor:

In [5]:
H = TTN(Hos, s)
@visualize H width=70 height=40

    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀H(2, 3)⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀(2)'⊗2⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀5⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀H(2, 2)⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

TreeTensorNetwork{Tuple{Int64, Int64}} with 12 vertices:
12-element Vector{Tuple{Int64, Int64}}:
 (1, 1)
 (2, 1)
 (3, 1)
 (4, 1)
 (1, 2)
 (2, 2)
 (3, 2)
 (4, 2)
 (1, 3)
 (2, 3)
 (3, 3)
 (4, 3)

and 11 edge(s):
(1, 1) => (2, 1)
(1, 1) => (1, 2)
(2, 1) => (3, 1)
(2, 1) => (2, 2)
(3, 1) => (4, 1)
(3, 1) => (3, 2)
(4, 1) => (4, 2)
(1, 2) => (1, 3)
(2, 2) => (2, 3)
(3, 2) => (3, 3)
(4, 2) => (4, 3)

with vertex data:
12-element Dictionary{Tuple{Int64, Int64}, Any}
 (1, 1) │ ((dim=8|id=65|"1×1,1×2"), (dim=8|id=746|"1×1,2×1"), (dim=2|id=54|"S=1…
 (2, 1) │ ((dim=8|id=746|"1×1,2×1"), (dim=8|id=705|"2×1,2×2"), (dim=8|id=894|"2…
 (3, 1) │ ((dim=8|id=894|"2×1,3×1"), (dim=8|id=644|"3×1,3×2"), (dim=8|id=95|"3×…
 (4, 1) │ ((dim=8|id=95|"3×1,4×1"), (dim=8|id=150|"4×1,4×2"), (dim=2|id=408|"S=…
 (1, 2) │ ((dim=5|id=819|"1×2,1×3"), (dim=8|id=65|"1×1,1×2"), (dim=2|id=621|"S=…
 (2, 2) │ ((dim=8|id=705|"2×1,2×2"), (dim=5|id=333|"2×2,2×3"), (dim=2|id=866|"S…
 (3, 2) │ ((dim=8|id=644|"3×1,3×2"), (dim=5|id=704

This gives a tensor network operator with a maximal bond dimension of 8 in this specific case.

In [6]:
linkdims(H)

DataGraph{Tuple{Int64, Int64}, Any, Int64, NamedGraph{Tuple{Int64, Int64}}, NamedEdge{Tuple{Int64, Int64}}} with 12 vertices:
12-element Vector{Tuple{Int64, Int64}}:
 (1, 1)
 (2, 1)
 (3, 1)
 (4, 1)
 (1, 2)
 (2, 2)
 (3, 2)
 (4, 2)
 (1, 3)
 (2, 3)
 (3, 3)
 (4, 3)

and 11 edge(s):
(1, 1) => (2, 1)
(1, 1) => (1, 2)
(2, 1) => (3, 1)
(2, 1) => (2, 2)
(3, 1) => (4, 1)
(3, 1) => (3, 2)
(4, 1) => (4, 2)
(1, 2) => (1, 3)
(2, 2) => (2, 3)
(3, 2) => (3, 3)
(4, 2) => (4, 3)

with vertex data:
0-element Dictionary{Tuple{Int64, Int64}, Any}

and edge data:
11-element Dictionary{NamedEdge{Tuple{Int64, Int64}}, Int64}
 (1, 1) => (2, 1) │ 8
 (1, 1) => (1, 2) │ 8
 (2, 1) => (3, 1) │ 8
 (2, 1) => (2, 2) │ 8
 (3, 1) => (4, 1) │ 8
 (3, 1) => (3, 2) │ 8
 (4, 1) => (4, 2) │ 8
 (1, 2) => (1, 3) │ 5
 (2, 2) => (2, 3) │ 5
 (3, 2) => (3, 3) │ 5
 (4, 2) => (4, 3) │ 5

### The Hamiltonian as a finite state machine

The automated `OpSum` to `TTN` conversion is performed by first constructing an intermediate symbolic sparse finite state machine representation of the Hamiltonian. This sparse representation is built in the form of a `DataGraph` with `SparseArrayKit.SparseArray`s as vertex data:

In [7]:
Hsparse, _ = ITensorNetworks.finite_state_machine(Hos, s, root_vertex)
typeof(Hsparse)

DataGraph{Tuple{Int64, Int64}, SparseArray{Sum{Scaled{Float64, Prod{Op}}}}, Any, NamedGraph{Tuple{Int64, Int64}}, NamedEdge{Tuple{Int64, Int64}}}

For each vertex, this sparse array encodes the operators acting on this vertex, and how they connect to other vertices:

In [8]:
Hsparse[2, 1]

14×14×17 SparseArray{Sum{Scaled{Float64, Prod{Op}}}, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  1.0 Id((2, 1),)
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0

In [9]:
nonzero_pairs(Hsparse[2, 1])

Dict{CartesianIndex{3}, Sum{Scaled{Float64, Prod{Op}}}} with 34 entries:
  CartesianIndex(1, 14, 7)   => 1.0 S+((2, 1),)
  CartesianIndex(1, 6, 17)   => 1.0 S+((2, 1),)
  CartesianIndex(6, 3, 17)   => 1.0 Id((2, 1),)
  CartesianIndex(1, 12, 15)  => 1.0 Id((2, 1),)
  CartesianIndex(4, 14, 2)   => 1.0 Id((2, 1),)
  CartesianIndex(1, 11, 14)  => 1.0 Id((2, 1),)
  CartesianIndex(8, 14, 17)  => 1.0 Sz((2, 1),)
  CartesianIndex(1, 14, 13)  => 2.0 Sz((2, 1),)
  CartesianIndex(1, 1, 17)   => 1.0 Id((2, 1),)
  CartesianIndex(1, 7, 17)   => -0.5 S-((2, 1),)
  CartesianIndex(1, 14, 10)  => 1.0 S-((2, 1),)
  CartesianIndex(1, 8, 17)   => 1.0 S-((2, 1),)
  CartesianIndex(1, 10, 17)  => 2.0 Sz((2, 1),)
  CartesianIndex(1, 14, 5)   => -0.5 S+((2, 1),)
  CartesianIndex(1, 13, 16)  => 2.0 Id((2, 1),)
  CartesianIndex(11, 14, 17) => 1.0 S-((2, 1),)
  CartesianIndex(14, 14, 17) => 1.0 Id((2, 1),)
  CartesianIndex(13, 14, 17) => 1.0 Sz((2, 1),)
  CartesianIndex(3, 2, 17)   => 1.0 Id((2, 1),)
  CartesianIn

## DMRG on a tree tensor networks

Given the Hamiltonian and some initial state, we can find the ground state using the DMRG algorithm. For example, we can run 10 sweeps of 2-site DMRG:

In [10]:
nsite = 2
nsweeps = 10
maxdim = 50
cutoff = 1e-5
outputlevel = 0

ψ = dmrg(H, ψ; nsweeps, maxdim, cutoff, nsite, outputlevel)
e = inner(ψ', H, ψ) / inner(ψ, ψ)
@show e;

e = -9.280045336117446 - 1.3327080450627432e-17im


We can compare this result to what we would get if we would used an MPS optimization instead by snaking an MPS along the comb tree to force it into a linear geometry.

In [11]:
snaked_vertices = snake_tree(c)
vmap = Dictionary(snaked_vertices, 1:length(snaked_vertices))
sline = map(v -> only(s[v]), snaked_vertices)
Hline = MPO(relabel_sites(Hos, vmap), sline)

ψline = randomMPS(sline; linkdims=χ)
eline, ψline = dmrg(Hline, ψline; nsweeps, maxdim, cutoff, nsite, outputlevel)
@show eline

@show abs(inner(ψ', H, ψ) - inner(ψline', Hline, ψline))

eline = -9.27968424885621
abs(inner(ψ', H, ψ) - inner(ψline', Hline, ψline)) = 0.00036108726122741075


0.00036108726122741075

As expected we get the exact same result. However, the MPS approach is naturally less compatible with the system geometry, which is evident from the fact that the MPO representation of the Hamiltonian has a larger bond dimension than its corresponding tree version:

In [12]:
# MPO
@show linkdims(Hline)

# TTN
@show collect(edge_data(linkdims(H)));

linkdims(Hline) = [5, 8, 8, 8, 11, 8, 8, 11, 8, 8, 5]
collect(edge_data(linkdims(H))) = [8, 8, 8, 8, 8, 8, 8, 5, 5, 5, 5]


## Upcoming

More features will be added to the tree sweeping framework in the near future, such as support for quantum number conservation, the use of custom dynamic sweeping patterns, and more.